In [49]:
##### Cleans FAO capital stock data
# aligns with core ISO codes and reformats data

import os
import pandas as pd

In [50]:
###### Load data

# Get the current working directory
cd = os.path.dirname(os.getcwd())

# Import data
capital_stock = pd.read_csv(f"{cd}/Data/Raw/FAO_capital_stock/FAOSTAT_capital_stock_01142026.csv")

country_codes = pd.read_csv(
    f"{cd}/Data/Correspondence_tables/country_names.csv",
    encoding="cp1252"
)

# Set save path
save_path = f"{cd}/Data/Clean/Capital_stock/FAO_capital_stock.csv"


In [51]:
##### Clean FAO data

# drop unnecessary columns
columns_to_keep = ['Area Code (M49)', 'Year', 'Value']
capital_stock = capital_stock[columns_to_keep]

# rename columns
capital_stock = capital_stock.rename(columns={
    'Area Code (M49)': 'FAO_code',
    'Year': 'Year',
    'Value': 'Values'
})

# reformat to wide format
capital_stock_wide = capital_stock.pivot(
    index='FAO_code',
    columns='Year',
    values='Values'
).reset_index()

# add units  
capital_stock_wide['Units'] = 'million USD (nominal)'

# merge with country codes to get ISO3 codes
capital_stock_wide = capital_stock_wide.merge(
    country_codes[['FAO_code', 'ISO3', 'Country_Name']],
    on='FAO_code',
    how='left'
)

# reorder columns
cols = ['ISO3', 'Country_Name', 'Units'] + [col for col in capital_stock_wide.columns if col not in ['ISO3', 'Country_Name', 'Units']]
capital_stock_wide = capital_stock_wide[cols]
capital_stock_wide = capital_stock_wide.drop(columns=['FAO_code'])


In [52]:
##### Save cleaned data

capital_stock_wide.to_csv(save_path, index=False)